In [ ]:
import csv
import pandas as pd
pd.options.plotting.backend = "plotly"
from os import listdir
from os.path import isfile, join
import plotly.graph_objects as go

In [ ]:
def generate_data(foldername):
    mean_data = {}
    stddev_data = {}
    filename = f"{foldername}/out.csv"
    results = pd.read_csv(filename, sep=';')
    for i, row in results.iterrows():
        desc = row['Descriptor']
        mode = row['Mode']
        if desc in mean_data:
            if mode in mean_data[desc]:
                mean_data[desc][mode] += [row['Mean']]
                stddev_data[desc][mode] += [row['StdDev']]
            else:
                mean_data[desc].update({mode: [row['Mean']]})
                stddev_data[desc].update({mode: [row['StdDev']]})
        else:
            mean_data[desc] = {mode: [row['Mean']]}
            stddev_data[desc] = {mode: [row['StdDev']]}


    return mean_data, stddev_data

In [ ]:

BENCHMARKS = [
              'batchnorm',
              'conv',
              'linear',
              'matmul',
              'wave'
             ]
base_path = "results/offload/"
CONFIGS = [    ##machine    , gpu
            ("e5_2698_v100", "V100"),
            ("gold_6130_rtx2080", "RTX2080"),
            ("i7_7700_GTX_1060", "GTX1060")
          ]
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

for bench in BENCHMARKS:
    fig_input = dict()
    adv_input = dict()
    err_input = dict()
    for machine, gpu in CONFIGS:
        machine_path = base_path + machine
        gcc_data, gcc_stddev = generate_data(f"{machine_path}/gcc_results/{bench}")
        clang_data, clang_stddev = generate_data(f"{machine_path}/clang_results/{bench}")
        descriptors = list(gcc_data.keys())
        for desc in descriptors:
            assert desc in clang_data
            if desc not in fig_input:
                fig_input[desc] = []
            if desc not in adv_input:
                adv_input[desc] = dict()
                err_input[desc] = dict()

            gcc_results = pd.DataFrame(data=gcc_data[desc])
            clang_results = pd.DataFrame(data=clang_data[desc])
            gcc_error = pd.DataFrame(data=gcc_stddev[desc])
            clang_error = pd.DataFrame(data=clang_stddev[desc])
            if "X" not in adv_input[desc]:
                adv_input[desc]["X"] = []

            adv_input[desc]["X"] += [f"{gpu}"]
            for index, key in enumerate(gcc_results):
                if f"{key}_GCC" not in adv_input[desc]:
                    adv_input[desc][f"{key}_GCC"] = []
                    adv_input[desc][f"{key}_CLANG"] = []
                    err_input[desc][f"{key}_GCC"] = []
                    err_input[desc][f"{key}_CLANG"] = []
                
                adv_input[desc][f"{key}_GCC"] += [gcc_results.iloc[0].values[index]]
                adv_input[desc][f"{key}_CLANG"] += [clang_results.iloc[0].values[index]]
                err_input[desc][f"{key}_GCC"] += [gcc_error.iloc[0].values[index]]
                err_input[desc][f"{key}_CLANG"] += [clang_error.iloc[0].values[index]]

    for desc, finput in fig_input.items():
        X_AXIS = adv_input[desc]["X"]
        Y_AXIS = {i:adv_input[desc][i] for i in adv_input[desc] if i != "X"}
        #ERR = {i:err_input[desc][i] for i in err_input[desc]}
        fig = go.Figure()
        for Y in Y_AXIS:
            fig.add_bar(x=X_AXIS, y=adv_input[desc][Y], name=Y,
                        error_y=dict(type='data', array=list(err_input[desc][Y]), visible=True))
        fig.update_layout({'title':f"{bench} - {desc}", 'yaxis': {'type': 'linear'}})
        fig.update_layout(
                        height=400,
                        width=600)
        fig.show()

